# Project 1: Linear Panel Data and Production Technology

*Authors*: Matias Piqueras, Sebastian Hørlück

This notebook contains the code to generate the output in *Project 1: Linear Panel Data and Production Technology*. The data, as given by the `read.ipynb` file, contains `N = 441` firms observed over `T = 12` years, 1968-1979. The variables are: 
* `lcap`: Log of capital stock, $k_{it}$ 
* `lemp`: log of employment, $\ell_{it}$ 
* `ldsa`: log of deflated sales, $y_{it}$
* `year`: the calendar year of the observation, `year` $ = 1968, ..., 1979$, 
* `firmid`: anonymized indicator variable for the firm, $i = 1, ..., N$, with $N=441$. 

In [1]:
#Add dir to import module because its in different dir. Change accordingingly.
import sys
import numpy as np
import pandas as pd
from panel import PlmFormula
from model_selection import wald_test, serial_correlation, hausman_test
%load_ext autoreload
%autoreload 2

#Import data and set multi-index
dat = pd.read_csv('firms.csv')
#Create leads
dat['lcap_lead'] = dat.groupby(['firmid'])['lcap'].shift(-1)
dat['lemp_lead'] = dat.groupby(['firmid'])['lemp'].shift(-1)
#Create lags
dat['lcap_lag'] = dat.groupby(['firmid'])['lcap'].shift(1)
dat['lemp_lag'] = dat.groupby(['firmid'])['lemp'].shift(1)

#Set multi-index
dat = dat.set_index(["firmid", "year"])

## Modeling: Fixed effects and first-difference

We estimated the following models bellow:
* `Fixed-Effects Estimator`
* `First-Difference Estimator`

We start by fixed effects estimated as

$$\ddot{y}_{i t} = \beta_{K} \ddot{k}_{i t} + \beta_{L} \ddot{\ell}_{i t} + \ddot{u}_{i t}$$

In [2]:
fe = PlmFormula(formula='ldsa ~ lcap + lemp', model="fe", data=dat,
                 include_intercept=False, cov_method = "robust").fit()

fe.summary()

            Results
_________________________________
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
Dependent variable: ldsa

      Beta         Se    t-values
----  --------  -----  ----------
lcap  0.155***  0.030       5.163
lemp  0.694***  0.042      16.667
_________________________________ 

R² = 0.477
Adj R² = 0.476
σ² = 0.018
Model: Fixed effects
No. observations: 5292
No. timeperiods: 12
_________________________________
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
Note: ∗p<0.1;∗∗p<0.05;∗∗∗p<0.01
Heteroscedastic robust standard errors.


Here we estimate

$$\Delta y_{i t} = \beta_{K} \Delta k_{i t} + \beta_{L} \Delta \ell_{i t} + \Delta u_{i t}$$

In [3]:
fd = PlmFormula(formula='ldsa ~ lcap + lemp', model="fd", data=dat,
                include_intercept=False, cov_method = "robust").fit()

fd.summary()

            Results
_________________________________
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
Dependent variable: ldsa

      Beta         Se    t-values
----  --------  -----  ----------
lcap  0.063***  0.023       2.746
lemp  0.549***  0.028      19.306
_________________________________ 

R² = 0.165
Adj R² = 0.164
σ² = 0.014
Model: First-difference
No. observations: 4851
No. timeperiods: 12
_________________________________
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
Note: ∗p<0.1;∗∗p<0.05;∗∗∗p<0.01
Heteroscedastic robust standard errors.


## Test for serial correlation

We test for serial correlation by running the following pooled ols based on the FD residuals

$$\hat{e}_{it} = \hat{\rho}_1 \hat{e}_{i,\, t-1} + \text{error}_{it}, \;\;\;\;\;\;\;\;
    t=3,4,\dots, T;i=1,2,\dots,N.$$

In [4]:
from model_selection import serial_correlation
#Extract the first-difference transformed data
X_fd, y_fd = fd._exog, fd._dependent
#Get len of unique timeperiods
t = len(np.unique([i[1] for i in dat.index]))
serial_correlation(y_fd, X_fd, t=t)

             Results
___________________________________
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
Dependent variable: OLS residual, eᵢₜ

       Beta          Se    t-values
-----  ---------  -----  ----------
eᵢₜ₋₁  -0.199***  0.015     -13.449
___________________________________ 

R² = 0.039
Adj R² = 0.039
σ² = 0.014
Model: Pooled OLS
No. observations: 4410
No. timeperiods: 1
___________________________________
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
Note: ∗p<0.1;∗∗p<0.05;∗∗∗p<0.01


## Wald-test of constant returns to scale

Here we test the following hypothesis using the wald-test

\begin{align}
    \mathcal{H}^{FD}_0 &: \beta_{k}^{FD} + \beta_{\ell}^{FD} = 1 \\
\label{eqn:fe_null}
    \mathcal{H}^{FE}_0 &: \beta_{k}^{FE} + \beta_{\ell}^{FE} = 1
\end{align}

where the test statistic is given by

$$    W = (R \mathbf{\hat{\beta}}-r)'(\widehat{\mathrm{Avar}}(\hat{\mathbf{\beta}}))^{-1}(R\mathbf{\hat{\beta}}-r)
$$

In [5]:
fd.results['b_hat']

array([[0.06296038],
       [0.54866574]])

In [6]:
fd.summary()

            Results
_________________________________
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
Dependent variable: ldsa

      Beta         Se    t-values
----  --------  -----  ----------
lcap  0.063***  0.023       2.746
lemp  0.549***  0.028      19.306
_________________________________ 

R² = 0.165
Adj R² = 0.164
σ² = 0.014
Model: First-difference
No. observations: 4851
No. timeperiods: 12
_________________________________
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
Note: ∗p<0.1;∗∗p<0.05;∗∗∗p<0.01
Heteroscedastic robust standard errors.


In [7]:
#Setup the test
R = np.ones(fd.results['b_hat'].shape[0]).reshape(1,-1)
r = 1
fd_W, fd_p = wald_test(fd.results['b_hat'], fd.results['cov'], R, r)
fe_W ,fe_p = wald_test(fe.results['b_hat'], fe.results['cov'], R, r)
print('First-difference\n----------------')
print(f"beta_k ({fd.results['b_hat'][0][0]:.4f}) = beta_l ({fd.results['b_hat'][1][0]:.4f}) -> p = {fd_p[0][0]:.4f}, W = {fd_W[0][0]:4.4f}")
print('\nFixed-effects\n--------------')
print(f"beta_k ({fe.results['b_hat'][0][0]:.4f}) = beta_l ({fe.results['b_hat'][1][0]:.4f}) -> p = {fe_p[0][0]:.4f}, W = {fe_W[0][0]:4.4f}")


First-difference
----------------
beta_k (0.0630) = beta_l (0.5487) -> p = 0.0000, W = 164.4890

Fixed-effects
--------------
beta_k (0.1546) = beta_l (0.6942) -> p = 0.0000, W = 19.4029


## Tests of strict exogenity

### Hausman

Because of limited space, the hausman test is not included in the paper. However, we include it here to show that the results are coherent with test including leads. The test statistic is given by

\begin{equation}
H=\left(\hat{\boldsymbol{\beta}}_{FE}-\hat{\boldsymbol{\beta}}_{FD}\right)^{\prime}\left[\mathrm{Avar}\left(\hat{\boldsymbol{\beta}}_{F E}\right)-\mathrm{Avar}\left(\hat{\boldsymbol{\beta}}_{FD}\right)\right]^{-1}\left(\hat{\boldsymbol{\beta}}_{FE}-\hat{\boldsymbol{\beta}}_{FD}\right)
\end{equation}

The test is robust, in the sense that we do not assume FE.3 and FD.3 under the null.

In [8]:
#Run hausman test
hausman_test(fe, fd, print_summary=True)

  b_fe    b_fd    b_diff
------  ------  --------
0.1546  0.0630    0.0917
0.6942  0.5487    0.1456
The Hausman test statistic is: 393.94, with p-value: 0.00.


### Including leads and lags

Here we include leads to test for strict exogeneity 

\begin{equation}
\ddot{y}_{i t} = \mathbf{\ddot{x}}_{i t}\mathbf{\beta} + \mathbf{\ddot{x}}_{i t+1}\mathbf{\delta} + \ddot{u}_{i t}
\end{equation}

In [9]:
#Define data for lead t+1
dat_lead = dat.dropna(subset=['lemp_lead', 'lcap_lead'])

fe_lead = PlmFormula(formula='ldsa ~ lcap + lemp + lcap_lead + lemp_lead', 
                      model="fe", data=dat_lead,include_intercept=False, cov_method = "robust").fit()

fe_lead.summary()

               Results
______________________________________
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
Dependent variable: ldsa

           Beta         Se    t-values
---------  --------  -----  ----------
lcap       0.028     0.038       0.746
lemp       0.541***  0.043      12.556
lcap_lead  0.167***  0.046       3.651
lemp_lead  0.142***  0.028       5.017
______________________________________ 

R² = 0.478
Adj R² = 0.478
σ² = 0.016
Model: Fixed effects
No. observations: 4851
No. timeperiods: 11
______________________________________
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
Note: ∗p<0.1;∗∗p<0.05;∗∗∗p<0.01
Heteroscedastic robust standard errors.
